In [1]:
from lonboard.experimental import ArcLayer

In [2]:
from lonboard import Map
import numpy as np

In [3]:
import requests

In [4]:
url = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/arc/counties.json"

In [5]:
r = requests.get(url)

In [6]:
source_data = r.json()

In [7]:
from matplotlib.colors import Normalize

In [8]:
arcs = []
targets = []
sources = []
pairs = {}

In [9]:
features = source_data['features']
for i, county in enumerate(features):
    flows = county['properties']['flows']
    target_centroid = county['properties']['centroid']
    total_value = {
        "gain": 0,
        "loss": 0,
    }

    for to_id, value in flows.items():
        if value > 0:
            total_value['gain'] += value
        else:
            total_value['loss'] += value

        # If number is too small, ignore it
        if abs(value) < 50:
            continue

        pair_key = '-'.join(map(str, sorted([i, int(to_id)])))
        source_centroid = features[int(to_id)]['properties']['centroid']
        gain = np.sign(flows[to_id])

        # add point at arc source
        sources.append({
            "position": source_centroid,
            "target": target_centroid,
            "name": features[int(to_id)]['properties']['name'],
            "radius": 3,
            "gain": -gain
        });

        # eliminate duplicate arcs
        if pair_key in pairs.keys():
            continue

        pairs[pair_key] = True

        if gain > 0:
            arcs.append({
                'target': target_centroid,
                'source': source_centroid,
                "value": flows[to_id],
            })
        else:
            arcs.append({
                'target': source_centroid,
                'source': target_centroid,
                "value": flows[to_id],
            })



    # add point at arc target
    targets.append({
        **total_value,
        "position": [target_centroid[0], target_centroid[1], 10],
        "net": total_value["gain"] + total_value["loss"],
        "name": county['properties']['name']
    });

# sort targets by radius large -> small
targets = sorted(targets, key=lambda d: abs(d['net']), reverse=True )
normalizer = Normalize(0, abs(targets[0]['net']))

In [10]:
value = np.array([arc['value'] for arc in arcs])
get_source_position = np.array([arc['source'] for arc in arcs])
get_target_position = np.array([arc['target'] for arc in arcs])
get_source_color = [166, 3, 3]
get_target_color = [35, 181, 184]
stroke_width = 1
opacity = 0.7

In [11]:
import pyarrow as pa

In [12]:
table = pa.table({
    'value': value
})

In [15]:
layer = ArcLayer(
    table=table,
    get_source_position=get_source_position,
    get_target_position=get_target_position,
    get_source_color=get_source_color,
    get_target_color = get_target_color,
    get_width = stroke_width,
    opacity=0.4,
)

In [16]:
map_ = Map(layers=[layer], _initial_view_state={})
map_

Map(layers=[ArcLayer(get_source_color=[166, 3, 3], get_source_position=<pyarrow.lib.FixedSizeListArray object …